In [13]:
#import bibliotek
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import xgboost as xgb
import missingno as msno

import xgboost as xgb
import missingno as msno
from tqdm import tqdm
import eli5


In [14]:
#aby widac wszyskie kolumny w df
pd.set_option('display.max_columns', 500)

In [15]:
#wczytanie danych
df = pd.read_hdf("../input/train_property.h5")
df.shape

(45694, 35)

In [16]:
cat_feats =  [x for x in df.columns if ":" in x]

In [17]:
for feat in tqdm(cat_feats) : 
    df["{}_cat".format(feat)] = df[feat].factorize()[0]

100%|██████████| 30/30 [00:00<00:00, 114.18it/s]


In [18]:
num_feats =  [x for x in df.columns if "_cat" in x]

In [19]:
#zbiory do modelu
X = df[num_feats].values
y = df["price"].values
y_log = np.log(y)

In [20]:
#parametry dla xgb
xgb_params = dict(
    max_depth=5, 
    n_estimators=50, 
    learning_rate=0.3, 
    random_state=0)

In [21]:
model = xgb.XGBRegressor(**xgb_params)
model.fit(X, y_log)
y_log_pred = model.predict(X)
y_pred = np.exp(y_log_pred)

In [22]:
mean_absolute_error(y, y_pred)

6.123177965294147

In [25]:
eli5.show_weights(model,feature_names= num_feats, top=40)

Weight,Feature
0.1999,Housing class:_cat
0.1477,Construction phase:_cat
0.1144,Building type:_cat
0.0782,Ad type:_cat
0.0643,Object type:_cat
0.0516,The view from the window:_cat
0.0464,Bathroom type:_cat
0.0361,Security:_cat
0.0332,Repair:_cat
0.0264,Phone:_cat


In [26]:
agg_funcs = [np.mean, np.median, np.std, np.size]
pd.pivot_table(df, index=["Housing class:"], values=["price"], aggfunc=agg_funcs)

,mean,median,std,size
,price,price,price,price
Housing class:,,,,
Business class,16.960212,13.870000,14.529679,13265.0
Comfort class,8.896281,8.220000,4.088762,18592.0
Economy class,9.014967,7.970000,5.578974,1167.0
Elite class,93.180874,59.735437,111.694224,2744.0


In [28]:
agg_funcs = [np.mean, np.median, np.std, np.size]
pd.pivot_table(df, index=["Housing class:","Construction phase:"], values=["price"], aggfunc=agg_funcs)

mean     median         std    size
                                         price      price       price   price
Housing class: Construction phase:                                           
Business class Building walls        15.807432  14.377584    8.108724  3907.0
               Commissioned          22.969876  16.070000   24.118084  3343.0
               Finish                15.338834  13.867500    8.387727  2433.0
               Is GK                 15.900000  15.660000    5.133008    67.0
               Pit                   10.329118   9.025000    4.524799  1842.0
               Playground            14.608252  13.790000    5.473676  1181.0
               Project               23.914671  20.400000   14.951960   492.0
Comfort class  Building walls         8.956350   8.303165    3.545199  9269.0
               Commissioned          11.818782  10.730000    6.031318   562.0
               Finish                 7.934991   7.261194    2.660985  4454.0
               Pit                    9.424779   8.845120    3.806229  1984.0
               Playground             9.046278   8.964000    3.277342  2060.0
               Project               11.652283  10.206881   17.472946   263.0
Economy class  Building walls         6.715802   6.410000    2.104886   243.0
               Commissioned           9.968217   8.160000    7.731570   497.0
               Finish                 5.891495   5.700000    2.108389    97.0
               Playground            10.190448  10.131960    2.468527   330.0
Elite class    Building walls        63.827085  53.660000   47.811529   341.0
               Commissioned         108.678063  68.692320  126.535385  1837.0
               Finish                94.028601  70.440000   83.129827   234.0
               Pit                   16.564800  16.155000    6.629807   100.0
               Playground            40.119276  23.600000   38.149889    87.0
               Project               49.187118  27.190000   56.478712   145.0